In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
from dataset import SegmentationDataset, SegmentationInferenceDataset
from data import *
from util import *

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

In [4]:
_, _, test_df = get_dfs()

In [5]:
test_dataset = SegmentationInferenceDataset(test_df, input_size=(128, 128), with_gt=False, with_raw_input=False, use_depth_channels=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

# First submission

In [7]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNet(1, n_classes=1)
weight = torch.load(str(model_dir / 'unet_test_dict.model'))
model.load_state_dict(weight)
model = model.to(device)

In [16]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [20]:
test_df['rle_mask'] = rles

In [28]:
submit(test_df, 'first_submission.csv')

# UNetRes submission

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(1, n_classes=1)
weight = torch.load(str(model_dir / 'unet_resblock_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

UNetRes(
  (inc): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


# UNetRes BCELoss SGD 3ch submission

In [11]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(n_channels=3, n_classes=1)
weight = torch.load(str(model_dir / 'unet_res_bceloss_sgd_3ch_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

UNetRes(
  (inc): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [12]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_bceloss_sgd_3ch_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


# UNetRes LovaszLoss SGD 3ch submission

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(n_channels=3, n_classes=1)
weight = torch.load(str(model_dir / 'unet_res_lovaszloss_sgd_3ch_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()


UNetRes(
  (inc): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_lovaszloss_sgd_3ch_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
